## 演習
- XORをpythonで実装し,正解を確認してください.

In [1]:
def AND(x, y):
    w_1 = 0.5
    w_2 = 0.5
    theta = 0.7
    return 1 if  w_1 * x + w_2 * y >= theta else 0

In [2]:
print(AND(1, 0), AND(0, 1), AND(0, 0), AND(1, 1))

0 0 0 1


In [3]:
def OR(x, y):
    w_1 = 0.5
    w_2 = 0.5
    theta = 0.3
    return 1 if  w_1 * x + w_2 * y >= theta else 0

In [4]:
print(OR(1, 0), OR(0, 1), OR(0, 0), OR(1, 1))

1 1 0 1


In [5]:
def NAND(x, y):
    w_1 = - 0.5
    w_2 = - 0.5
    theta = - 0.7
    return 1 if  w_1 * x + w_2 * y >= theta else 0

In [6]:
print(NAND(1, 0), NAND(0, 1), NAND(0, 0), NAND(1, 1))

1 1 1 0


In [7]:
def XOR(x, y):
    return AND(NAND(x, y), OR(x, y))

In [8]:
print(XOR(1, 0), XOR(0, 1), XOR(0, 0), XOR(1, 1))

1 1 0 0


## 演習
以下を微分せよ.ただし、$ReLU$は$x=0$では劣微分を求めてください.
- sigmoid :$\sigma(x) = \frac{1}{1 + e^{-x}}$
- ReLU: $ReLU(x) = \max\\{x, 0\\}$

### 回答
- $(1-\sigma(x))\sigma(x)$
- ReLU
  - $1 (x > 0)$
  - $0 (x < 0)$
  - $[0, 1] x = 0$
## 演習 2
- 3層のNNのクラスを実装せよ
  - 入力3次元
  - 出力4次元

In [9]:
import torch
import torch.nn as nn

In [10]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(3, 5)
        self.l2 = nn.Linear(5, 4)

    def forward(self, x):
        y = self.l1(x)
        y = self.l2(y)
        return y

In [11]:
net = Net()
x = torch.arange(3).float()
net(x)

tensor([-0.7255,  0.2169,  0.0748, -0.4225], grad_fn=<AddBackward0>)

## 演習
実際に3層のNNに対して,誤差逆伝搬を使い,損失の微分を計算しよう.
- 活性化関数は全て$sigmoid$
- 入力(1,1), 出力(1,0)
- 損失は二乗誤差
$$
W_1 = \begin{pmatrix}
2 & 3 \\
1 & 4 \\
\end{pmatrix}
$$
$$
W_2 = \begin{pmatrix}
3 & -1 \\
0 & 4 \\
\end{pmatrix}
$$

In [14]:
W1 = torch.tensor([[2., 3.], [1., 4.]], requires_grad=True)
W2 = torch.tensor([[3., -1.], [0., 4.]], requires_grad=True)

In [16]:
x = torch.tensor([1., 1.], requires_grad=True)
y = torch.tensor([1., 0.])

In [22]:
from torch.functional import F
sigma = F.sigmoid
mse = nn.MSELoss()

In [25]:
# 出力の計算
o = F.linear(x, W1)
o = sigma(o)
o = F.linear(o, W2)
o = sigma(o)

In [26]:
# 損失
loss = mse(o, y)

In [28]:
torch.autograd.grad(loss, x, retain_graph=True)

(tensor([4.7808e-05, 1.4670e-03]),)

## 演習
- $W$は2×2の行列
- $x,y$は2次元のベクトル
- $loss = |y - Wx|^2$の時
- $x,y, W$から$loss$までを計算する計算グラフを書きましょう
- 計算グラフに従って微分の計算をしてみてください


## 回答
- 手計算が想定回答ですが、実装するなら

In [66]:
W = torch.tensor([[2., 3.], [1., 4.]], requires_grad=True)
x = torch.tensor([1., 1.], requires_grad=True)
y = torch.tensor([1., 0.])

In [67]:
z = F.linear(x, W)
loss = mse(z, y)

In [61]:
print(loss.grad_fn)
print("squeezeは不要な次元を削除", loss.grad_fn.next_functions, )

print("行列の掛け算", loss.grad_fn.next_functions[0][0].next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions[0][0].next_functions)
# Unsquessze 型の変換
# T: 転置
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]\
.next_functions[0][0])
print("accumulateは累積和", loss.grad_fn.next_functions[0][0].next_functions[0][0]\
.next_functions[0][0].next_functions[0][0])

squeezeは不要な次元を削除 ((<SqueezeBackward3 object at 0x7f5f9a54c290>, 0),)
行列の掛け算 ((<MmBackward object at 0x7f5f9a634390>, 0),)
((<UnsqueezeBackward0 object at 0x7f5f99af0390>, 0), (<TBackward object at 0x7f5f99af0e50>, 0))
accumulateは累積和 <AccumulateGrad object at 0x7f5f9af78ed0>


In [63]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]\
.next_functions[0][0].next_functions[0][0](torch.tensor([1, 2]))

()

In [43]:
loss.grad_fn.next_functions[0][0].next_functions[0]

((<MmBackward at 0x7f5f9a634390>, 0),)

In [69]:
loss.backward()

In [70]:
x.grad

tensor([13., 32.])

In [72]:
W.grad

tensor([[4., 4.],
        [5., 5.]])

## 演習
- Pytorchで(3層の)DNNを使い`iris`の分類を行ってください.
- 損失関数はcross_entropyを使ってください.
- optimzierをSGDで学習してください

In [73]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

# データの分割
X_train, X_valid, y_train, y_valid = train_test_split(iris.data, iris.target, test_size=0.2)

class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, index):
        
        data = self.data[index]
        label = self.labels[index]
        
        if not self.transform is None:
            data, label = self.transform(data, label)
        
        return data, label
def iris_transform(data, label):
    # inputはdoubleをfloatにしている.
    return torch.tensor(data).float(), torch.tensor(label)

iris = load_iris()
X_train, X_valid, y_train, y_valid = train_test_split(iris.data, iris.target, test_size=0.2)

iris_train_dataset = IrisDataset(X_train, y_train, iris_transform)
iris_valid_dataset = IrisDataset(X_valid, y_valid, iris_transform)

In [106]:
batch_size  = 30 # ミニバッチのデータの数
train_loader = torch.utils.data.DataLoader(iris_train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(iris_valid_dataset, batch_size=batch_size, shuffle=True)

In [107]:
import torch.nn.functional as F

In [108]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(4, 3)
        self.l2 = nn.Linear(3, 3)

    def forward(self, x):
        out = self.l1(x)
        out = F.relu(out)
        y = self.l2(out)
        return y

In [109]:
model = Net()

max_epoch = 100 #

criterion = nn.CrossEntropyLoss() # 損失の定義
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #(確率的)勾配降下法

In [110]:
def train_epoch(model, data_loader, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader): # 入力と正解
         optimizer.zero_grad() # Weightの初期化
         output = model(data) # 仮説で値代入
         output.dtype
         loss = criterion(output, target) # 損失
         loss.backward() # 微分の計算
         optimizer.step() # パラメータの更新
         
    if epoch % 10 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            batch_idx, batch_idx * len(data), len(data_loader.dataset),
            100. * batch_idx / len(data_loader), loss.item()))

def valid_epoch(model, data_loader, epoch):
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader): # 入力と正解
             optimizer.zero_grad() # Weightの初期化
             output = model(data) # 仮説で値代入
             output.dtype
             loss = criterion(output, target) # 損失
            
        if epoch % 10 == 0:
             print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                 batch_idx, batch_idx * len(data), len(data_loader.dataset),
                 100. * batch_idx / len(data_loader), loss.item()))

In [114]:
for epoch in range(100):
    train_epoch(model, train_loader, epoch)
    valid_epoch(model, valid_loader, epoch)

Train Epoch: 3 [90/120 (75%)]	Loss: 0.166712
Test Epoch: 0 [0/30 (0%)]	Loss: 0.080890
Train Epoch: 3 [90/120 (75%)]	Loss: 0.176670
Test Epoch: 0 [0/30 (0%)]	Loss: 0.079282
Train Epoch: 3 [90/120 (75%)]	Loss: 0.068848
Test Epoch: 0 [0/30 (0%)]	Loss: 0.077608
Train Epoch: 3 [90/120 (75%)]	Loss: 0.044793
Test Epoch: 0 [0/30 (0%)]	Loss: 0.080907
Train Epoch: 3 [90/120 (75%)]	Loss: 0.022234
Test Epoch: 0 [0/30 (0%)]	Loss: 0.080694
Train Epoch: 3 [90/120 (75%)]	Loss: 0.032296
Test Epoch: 0 [0/30 (0%)]	Loss: 0.080627
Train Epoch: 3 [90/120 (75%)]	Loss: 0.039864
Test Epoch: 0 [0/30 (0%)]	Loss: 0.083666
Train Epoch: 3 [90/120 (75%)]	Loss: 0.050364
Test Epoch: 0 [0/30 (0%)]	Loss: 0.079047
Train Epoch: 3 [90/120 (75%)]	Loss: 0.040048
Test Epoch: 0 [0/30 (0%)]	Loss: 0.078542
Train Epoch: 3 [90/120 (75%)]	Loss: 0.056868
Test Epoch: 0 [0/30 (0%)]	Loss: 0.078962


## 演習
- Pytorchで(3層の)DNNを使い`iris`の分類を行ってください.
- 損失関数はcross_entropyを使ってください.
- optimzierをAdam/RMSPropでも学習してみてください

## 演習
- PytorchでDNNを使いMNISTの分類を行ってください.
ただし、以下は行ってください.
- weight_decay
- Dropout
- BatchNormalization

In [128]:
from torchvision.datasets import MNIST
from torchvision import transforms

In [121]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    MNIST('./mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

In [191]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        # 行列を定義
        self.l1 = nn.Linear(input_size, input_size)
        self.l2 = nn.Linear(input_size, output_size)
        self.bm = nn.BatchNorm1d(input_size)
        self.do = nn.Dropout(p=0.2)
    
    def forward(self, x):
        x = x.view(-1, self.input_size) # reshape
        x = self.bm(x)
        x = F.relu(self.l1(x))
        x = self.do(x)
        return self.l2(x)

In [192]:
model = Net(28*28, 10)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.01)
optimizer.zero_grad()
criterion = nn.CrossEntropyLoss()
optimizer.step()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [196]:
def train(model, train_loader):
    model.train()
    tot_loss = 0
    for id, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        if id % 300 == 0:
            print("loss", tot_loss)
            pred = out.argmax(axis=1)
            print("acc", (pred == labels).sum().item() / labels.shape[0])

In [195]:
model.train()
tot_loss = 0
for id, (data, labels) in enumerate(train_loader):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, labels)
    loss.backward()
    tot_loss += loss.item()
    optimizer.step()
    if id % 300 == 0:
        print("loss", tot_loss)
        pred = out.argmax(axis=1)
        print("acc", (pred == labels).sum().item() / labels.shape[0])

loss 2.0840952396392822
acc 0.28125
loss 143.36962301284075
acc 0.90625
loss 240.52823129296303
acc 0.9375
loss 334.60667476058006
acc 0.90625
loss 428.2117058336735
acc 0.96875
loss 526.5103172361851
acc 0.9375
loss 629.6539689004421
acc 0.96875
